In [1]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
import pandas as pd
import itertools
import timeit
import json

FILENAME = './datasets/coco.json'

#### 2.2.1 Implement a custom version of Apriori

In [2]:
def generateCandidatesByJoin(candidates):
    """
    Apriori candidates generation using join (and not cartisian product)
    """
    candidates = [list(x) for x in list(candidates)]
    candidates_without_last_item = [x[:-1] for x in candidates]
    
    new_candidates = []
    
    for i in range(0, len(candidates)):
        for j in range(i+1, len(candidates)):
            if(candidates_without_last_item[i] == candidates_without_last_item[j]):
                new_candidate = list(set(candidates[i] + candidates[j]))
                new_candidate.sort()
                new_candidates.append(new_candidate)
    
    new_candidates_dict = {}
    
    for new_candidate in new_candidates:
        new_candidates_dict[tuple(new_candidate)] = {
            'support_count': 0,
            'support': 0,
        }
    
    return new_candidates_dict

def my_apriori(itemsets, minsup):
    """
    Get frequent itemsets using the Apriori algorithm
    
    Input:
    - itemsets: list of lists. List of transactions with list of items for each transaction.
    - minsup: float. The relative support (absolute support / number of transactions).
    
    Output: dictionary. Key: tuple of itemset. Values: 'support' and 'support_count'.
    """
    
    transactions_number = len(itemsets)
    
    # First of all, sort each list of elements for each transaction
    for itemset in itemsets:
        itemset.sort()
    
    """Generate candidates"""

    # Frequent items (fi) is a dictionary
    # Each row contains:
    #     - Key:
    #       - a frozenlist of the itemset
    #     - Values:
    #       - the support count (support_count)
    #       - the support (support)
    fi = {}
    
    # Candidates has the same structure of frequent items
    candidates = {}
    
    # Generate frequent itemsets for k = 1 (first iteration)
    for itemset in itemsets:
        for item in itemset:
            item = (item,)
            
            if item in candidates:
                candidates[item]['support_count'] += 1
                candidates[item]['support'] = candidates[item]['support_count'] / transactions_number
            else:
                candidates[item] = {
                    'support_count': 1,
                    'support': 1 / transactions_number,
                }
    
    # Pruning - Keep just the candidates with support >= minsup
    previous_candidates = {k: v for k, v in candidates.items() if v['support'] >= minsup}
    fi = previous_candidates
    
    # Generate frequent itemsets for k = 1 (cartisian product)
    candidates = {}
    
    # Generate the combination of elements
    list_of_lists_for_combinations = [list(x) for x in list(previous_candidates)]
    list_for_combinations = []
    for element in list_of_lists_for_combinations:
        list_for_combinations += element
    candidates_list = list(itertools.combinations(list_for_combinations, 2))

    for candidate in candidates_list:
        candidates[candidate] = {
            'support_count': 0,
            'support': 0,
        }
    
    # Generate frequent itemsets for k >= 2 (join)
    k = 0
    while True:
        
        """Compute the support for the candidates"""
        
        # For each transaction
        for itemset in itemsets:
            
            # For each candidate (set of items) generated at the previous iteration
            for candidate_key, candidate_val in candidates.items():
                candidate_in_current_transaction = True
                
                # If at least one of the items in the candidate set is not in the current transaction, prune it
                for candidate_item in list(candidate_key):
                    if not candidate_item in itemset:
                        candidate_in_current_transaction = False
                    
                # If the candidate is present, increment the support and the support count
                if candidate_in_current_transaction:
                    candidates[candidate_key]['support_count'] += 1
                    candidates[candidate_key]['support'] = candidates[candidate_key]['support_count'] / transactions_number
        
        # Pruning - Keep the candidates with (support >= minsup) only
        previous_candidates = {k: v for k, v in candidates.items() if v['support'] >= minsup}     
        fi.update(previous_candidates)
        
        if len(previous_candidates) == 0:
            # No more frequent items with the current cardinality
            # This means that we won't have any frequent item with higher cardinalities
            # Therefore, we can stop
            break
        
        else:
            candidates = generateCandidatesByJoin(previous_candidates)

    """Transform current data structure to a Pandas DataFrame"""
    # 1. Convert the dictionary to a list
    fi_list = list(fi.items())

    # 2. Generate the input data structured in the proper way
    df_input = []
    for item in fi_list:
        df_input.append([item[1]['support'], frozenset(item[0]), len(item[0])])
    
    # 3. Generate the DataFrame
    fi = pd.DataFrame(df_input, columns = ['support', 'itemsets', 'length'])
    
    # 4. Sort the data for DESC values of 'support'
    fi = fi.sort_values('support', ascending = False)
    
    # 5. Reassign rows index after the having sorted
    fi.reset_index(drop=True, inplace=True)
    
    return fi

#### 2.2.2 Load the COCO dataset

In [3]:
with open(FILENAME) as f:
    coco_images = json.load(f)
    
# Itemsets is a list of lists
# It contains a list of elements for each transaction
itemsets = []

for image in coco_images:
    itemsets.append(image['annotations'])

#### 2.2.3 Run the custom Apriori implementation

In [4]:
minsup = 0.1

fi = my_apriori(itemsets, minsup)

display(fi[fi['length'] > 1])

,support,itemsets,length
4,0.3208,"(bench, person)",2
7,0.2386,"(car, person)",2
9,0.1978,"(car, traffic light)",2
10,0.1902,"(traffic light, person)",2
16,0.1362,"(car, traffic light, person)",3
18,0.1224,"(handbag, person)",2
22,0.1032,"(car, truck)",2


#### 2.2.4 Convert the dataset to fit the mlxtend.apriori input and execute it

In [5]:
dataset = []

for image in coco_images:
    dataset.append(image['annotations'])
    
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

df = pd.DataFrame(te_ary, columns=te.columns_)

In [6]:
# mlxtend Apriori

fi_apriori = apriori(df, min_support=minsup, use_colnames=True)
fi_apriori['length'] = fi_apriori['itemsets'].apply(lambda x: len(x))

fi_apriori[(fi_apriori['length'] >= 2)]

,support,itemsets,length
8,0.3208,"(bench, person)",2
9,0.2386,"(car, person)",2
10,0.1978,"(car, traffic light)",2
11,0.1032,"(car, truck)",2
12,0.1224,"(handbag, person)",2
13,0.1902,"(traffic light, person)",2
14,0.1362,"(car, traffic light, person)",3


In [7]:
# mlxtend FP-Growth

fi_fpgrowth = fpgrowth(df, min_support=minsup, use_colnames=True)
fi_fpgrowth['length'] = fi_fpgrowth['itemsets'].apply(lambda x: len(x))

fi_fpgrowth[(fi_fpgrowth['length'] >= 2)]

,support,itemsets,length
8,0.2386,"(car, person)",2
9,0.3208,"(bench, person)",2
10,0.1032,"(car, truck)",2
11,0.1978,"(car, traffic light)",2
12,0.1902,"(traffic light, person)",2
13,0.1362,"(car, traffic light, person)",3
14,0.1224,"(handbag, person)",2


#### 2.2.5 Execution time

In [8]:
minsup = 0.01

print("FP-Growth")
display(timeit.timeit(lambda: fpgrowth(df, minsup), number=1))

print("Apriori")
display(timeit.timeit(lambda: apriori(df, minsup), number=1))

print("My Apriori")
display(timeit.timeit(lambda: my_apriori(itemsets, minsup), number=1))

FP-Growth


0.06486009999999975

Apriori


0.1262732999999998

My Apriori


6.9963374

With minsup = 0.001 we obtain:

    FP-Growth:   0.108402400000017
    Apriori:     1.946937600000012
    My Apriori: 53.637261600000016

#### 2.2.6 Exhaustive search

In [9]:
def generateItemsetsLessEqLen(itemsets, k):
    """
    Generates all possible itemsets of length less than or equal to k
    
    Input: - itemset: list.
           - k: int
    Output: list of lists
    """
    if k < 1:
        raise "k must be at least 1"
    
    i = 1
    combinations = []
    
    for i in range(1, k+1):
        combinations += itertools.combinations(itemsets, i)
    
    return combinations

def getSupportCount(combinations):
    
    supports = {}
    
    for combination in combinations:
        supports[combination] = 0
    
    # Complexity should be:
    # O(N * K * N * K) = O( N^2 * K^2 )
    # with - N: number of images
    #      - K: number of combinations (factorial on the number of labels)
    
    for itemset in itemsets:
        for combination in combinations:
            if all(comb_item in itemset for comb_item in combination):
                supports[combination] += 1
    
    return supports

In [10]:
k = 1

all_labels = []
for item in itemsets:
    all_labels += item
all_labels = list(set(all_labels))

combinations = generateItemsetsLessEqLen(all_labels, k)

display(timeit.timeit(lambda: getSupportCount(combinations), number=1))

0.3122392999999999

    k = 1 →   0.2669683999997
    k = 2 →   9.3415089000000
    k = 3 → 222.8402719000000

Time is NP. This algorithm would not be feasible.